In [1]:
import pandas as pd
from datetime import datetime

df = pd.read_csv("C:/Users/karth/OneDrive/Desktop/retail-forecast-segmentation-inventory/data/raw/Superstore.csv",
                 encoding="latin1")
df.head()


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


#RFM Table

In [2]:
# Convert Order Date just in case
df['Order Date'] = pd.to_datetime(df['Order Date'])

# Reference date = max date in data + 1 day (for recency)
ref_date = df['Order Date'].max() + pd.Timedelta(days=1)

rfm = df.groupby('Customer ID').agg({
    'Order Date': lambda x: (ref_date - x.max()).days,   # Recency
    'Product ID': 'nunique',                             # Frequency = unique products or orders
    'Sales': 'sum'                                       # Monetary
}).reset_index()

rfm.rename(columns={
    'Order Date': 'Recency',
    'Product ID': 'Frequency',
    'Sales': 'Monetary'
}, inplace=True)

rfm.head()


,Customer ID,Recency,Frequency,Monetary
0,AA-10315,185,11,5563.560
1,AA-10375,20,15,1056.390
2,AA-10480,260,12,1790.512
3,AA-10645,56,18,5086.935
4,AB-10015,416,6,886.156


In [3]:
# Helper to score based on quantiles
def r_score(x, q):
    if x <= q[0.25]:
        return 4
    elif x <= q[0.50]:
        return 3
    elif x <= q[0.75]:
        return 2
    else:
        return 1

def fm_score(x, q):
    if x <= q[0.25]:
        return 1
    elif x <= q[0.50]:
        return 2
    elif x <= q[0.75]:
        return 3
    else:
        return 4

r_quartiles = rfm['Recency'].quantile([0.25, 0.5, 0.75])
f_quartiles = rfm['Frequency'].quantile([0.25, 0.5, 0.75])
m_quartiles = rfm['Monetary'].quantile([0.25, 0.5, 0.75])

rfm['R_Score'] = rfm['Recency'].apply(lambda x: r_score(x, r_quartiles))
rfm['F_Score'] = rfm['Frequency'].apply(lambda x: fm_score(x, f_quartiles))
rfm['M_Score'] = rfm['Monetary'].apply(lambda x: fm_score(x, m_quartiles))

rfm['RFM_Score'] = rfm['R_Score'] + rfm['F_Score'] + rfm['M_Score']

rfm.head()


,Customer ID,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score
0,AA-10315,185,11,5563.560,1,2,4,7
1,AA-10375,20,15,1056.390,4,3,1,8
2,AA-10480,260,12,1790.512,1,2,2,5
3,AA-10645,56,18,5086.935,3,4,4,11
4,AB-10015,416,6,886.156,1,1,1,3


# K-Means clustering on RFM

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

features = rfm[['Recency', 'Frequency', 'Monetary']].copy()

scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# 4 clusters as a good default
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
rfm['Cluster'] = kmeans.fit_predict(scaled_features)

rfm.head()


,Customer ID,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score,Cluster
0,AA-10315,185,11,5563.560,1,2,4,7,0
1,AA-10375,20,15,1056.390,4,3,1,8,0
2,AA-10480,260,12,1790.512,1,2,2,5,1
3,AA-10645,56,18,5086.935,3,4,4,11,0
4,AB-10015,416,6,886.156,1,1,1,3,2


In [6]:
# Map clusters to segment labels based on RFM_Score average
cluster_summary = rfm.groupby('Cluster')['RFM_Score'].mean().sort_values()
cluster_summary


Cluster
2     4.409524
1     6.074713
0     9.638989
3    10.174603
Name: RFM_Score, dtype: float64

In [7]:
# Adjust mapping based on your output; here we assume:
# lowest RFM_Score cluster index -> "At Risk"
# mid -> "Need Attention"
# high -> "Loyal"
# highest -> "Champions"

# Sort clusters by average RFM score
ordered_clusters = cluster_summary.index.tolist()

segment_map = {
    ordered_clusters[0]: "At Risk",
    ordered_clusters[1]: "Need Attention",
    ordered_clusters[2]: "Loyal",
    ordered_clusters[3]: "Champions"
}

rfm['Segment'] = rfm['Cluster'].map(segment_map)

rfm[['Customer ID', 'Recency', 'Frequency', 'Monetary', 'RFM_Score', 'Segment']].head()


,Customer ID,Recency,Frequency,Monetary,RFM_Score,Segment
0,AA-10315,185,11,5563.560,7,Loyal
1,AA-10375,20,15,1056.390,8,Loyal
2,AA-10480,260,12,1790.512,5,Need Attention
3,AA-10645,56,18,5086.935,11,Loyal
4,AB-10015,416,6,886.156,3,At Risk


In [8]:
output_path = "C:/Users/karth/OneDrive/Desktop/retail-forecast-segmentation-inventory/data/processed/customer_segments.csv"
rfm.to_csv(output_path, index=False)
output_path


'C:/Users/karth/OneDrive/Desktop/retail-forecast-segmentation-inventory/data/processed/customer_segments.csv'